In [2]:
import sys
import os

current_dir = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

In [3]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
from models.MemberProductAccount import MemberProductAccount
from models.MemberLevelScore import MemberLevelScore
from models.Level import Level
from models.StandardChartData import StandardDataPoint, StandardChartData
from models.Timeline import Timeline
from models.LevelsFull import LevelsFull, LevelData, Movement


In [ ]:
def load_levels(file_path):

    df = pd.read_csv(file_path)
    
    # levels = []

    # for i, row in df.iterrows():
        
    #     # Omits 'timestamp' since it is not a dataclass attribute
    #     level = Level(
    #         client_account_id=row['client_account_id'],
    #         level_id=row['level_id'],
    #         level_name=row['level_name'],
    #         level_score_start=row['level_score_start'],
    #         level_score_end=row['level_score_end']
    #     )

    #     print(level)
        
    #     levels.append(level)

    # return levels

    return df



levels = load_levels('../data/levels.csv')

In [24]:
# def load_member_level_scores(file_path):
#     # Read the CSV file with headers
#     df = pd.read_csv(file_path)  # Assuming the first row contains headers

#     # Define a function to create a MemberLevelScore instance from a row
#     def create_member_level_score(row):
#         return MemberLevelScore(
#             client_account_id=row['client_account_id'],  # Adjust based on CSV column names
#             member_id=row['member_id'],
#             level_score_type=row['level_score_type'],
#             level_score=row['level_score'],
#             active_member=row['active_member'],
#             score_date=row['score_date'],  # Optional
#             timestamp=row['timestamp']       # Optional
#         )

#     # Use apply to create instances of MemberLevelScore
#     member_level_scores = df.apply(create_member_level_score, axis=1)

#     return member_level_scores.tolist()  # Convert to list if needed

def load_member_level_scores(file_path):

    df = pd.read_csv(file_path)  # Assuming the first row contains headers

    # member_level_scores = df.apply(lambda row: MemberLevelScore(**row), axis=1)

    # return member_level_scores

    return df

member_level_scores = load_member_level_scores('../data/member_level_scores_history.csv')
print(member_level_scores[:5])

  client_account_id                timestamp  member_id level_score_type  \
0        federal-cu  2024-10-19 00:00:00.000       1769           MPP V4   
1        federal-cu  2024-10-19 00:00:00.000       1785           MPP V4   
2        federal-cu  2024-10-19 00:00:00.000       1849           MPP V4   
3        federal-cu  2024-10-19 00:00:00.000       1857           MPP V4   
4        federal-cu  2024-10-19 00:00:00.000       1881           MPP V4   

   level_score  score_date  active_member  
0    42.897669  2024-10-19           True  
1    11.825811  2024-10-19           True  
2    22.695028  2024-10-19           True  
3    42.872833  2024-10-19           True  
4    16.456328  2024-10-19           True  


In [21]:
def load_member_product_accounts(file_path):

    df = pd.read_csv(file_path)

    # member_product_accounts = df.apply(lambda row: MemberProductAccount(**row), axis=1)

    # return member_product_accounts

    return df

member_product_accounts = load_member_product_accounts('../data/member_product_accounts.csv')
print(member_product_accounts[:5])

  client_account_id                timestamp  account_id  \
0        federal-cu  2024-12-16 00:00:00.000     1630953   
1        federal-cu  2024-12-16 00:00:00.000     1270245   
2        federal-cu  2024-12-16 00:00:00.000     2680285   
3        federal-cu  2024-12-16 00:00:00.000     2680285   
4        federal-cu  2024-12-16 00:00:00.000     2680285   

              member_product_account_id  member_id  product_id  \
0  02325e1d-f8a0-41d0-9b90-1a272f113908       1769           0   
1  58fc84eb-678b-4517-a57f-2fcbdd91d2ce     189761        3000   
2  c7ee6e69-0899-4640-bc13-f0f6df33b9ae      28369        3011   
3  bd96899b-847a-4996-9325-9e6377e17229      28369        3011   
4  78139254-83a0-42ac-b5ae-11a7bdf54dfb      28369        3011   

  product_category_id        account_open_date       account_close_date  \
0     Primary Savings  1995-01-19 00:00:00.000                      NaN   
1       Personal Loan  1991-03-25 00:00:00.000                      NaN   
2       Personal 

In [ ]:
def create_levels_full(levels, member_level_scores, member_product_accounts):

    levels_data = []

    for level in levels:

        level_members = []

        for level in levels:
            level_members = [member for member in member_level_scores if level.level_score_start <= member['level_score'] <= level.level_score_end]
            member_count = len(level_members)

            level_members_ids = [level.member_id for level in level_members]
            level_member_products = [product for product in member_product_accounts if member_product_accounts['member_id'] in level_members_ids]
            if len(level_members) > 0:
                avg_product_count = len(level_member_products) / len(level_members)
            else:
                avg_product_count = 0
            
            